#### Libraries

In [ ]:
from functions import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from nilearn import image
from nilearn import datasets, surface
from nilearn import plotting
from matplotlib.colors import LinearSegmentedColormap

#### Directories and parameters

In [ ]:
results_dir = '../results/scores/'
results_dir_maps = '../results/maps/'
results_file_train = 'classification_performance_train_cv_5_rev.xlsx'
results_file_test = 'classification_performance_test_rev.xlsx'
results_file_features = 'classification_features_importances_test_rev.xlsx'
figures_dir = '../figures/individual/'
classifiers = ['LR', 'LDA', 'NB', 'DT', 'RF', 'XGB', 'GB', 'SVC', 'KNN']
metrics = ['Accuracy', 'F1', 'Recall']
mask_img = nib.load('../data/msk/sum_80_bin.nii')
measures=['GCOR', 'ECM_deg', 'ECM_norm', 'ECM_rank', 'fALFF', 'IHC']

#### Classification performance (Figure 2A, 2B)

In [ ]:
fig = plot_auc_heatmap(results_dir + results_file_test, classifiers*10)
fig.savefig(figures_dir + 'heatmap_test_rev.svg')

fig = plot_auc_heatmap(results_dir + results_file_train, classifiers*10)
fig.savefig(figures_dir + 'heatmap_train_rev.svg')

#### Classification performances (Figure 2C, 2D)

In [ ]:
fig = plot_metrics(results_dir + results_file_test, metrics, classifiers)
fig.savefig(figures_dir + 'dotplot_test_rev.svg')

fig = plot_metrics(results_dir + results_file_train, metrics, classifiers)
fig.savefig(figures_dir + 'dotplot_train_rev.svg')

#### Critical difference diagrams (Figure 3A, 3B)

In [ ]:
fig, fig2 = plot_critical_differences(results_dir + results_file_test, results_dir + 'Friedmann_test_rev.txt', classifiers, 'test_rev.svg', figures_dir)

fig, fig2 = plot_critical_differences(results_dir + results_file_train, results_dir + 'Friedmann_train_rev.txt', classifiers, 'train_rev.svg', figures_dir)

#### Boxplot ROCAUC (Figure 3C, 3D)

In [ ]:
fig = plot_boxplots(results_dir + results_file_test, classifiers)
fig.savefig(figures_dir + 'violin_plot_test_rev.svg')

fig = plot_boxplots(results_dir + results_file_train, classifiers)
fig.savefig(figures_dir + 'violin_plot_train_rev.svg')

#### Generate maps for Figure 4

In [ ]:
for measure in measures:
    print(f"Processing feature importance for measure: {measure}")
    df_feat = pd.read_excel(results_dir + results_file_features, sheet_name=measure)

    for model_name in df_feat.columns:
        importance_scores = df_feat[model_name].values

        tmp_img = nilearn.masking.unmask(importance_scores, mask_img)

        nii_data = tmp_img.get_fdata()
        nii_affine = tmp_img.affine
        importance_nii = nib.Nifti1Image(nii_data, affine=nii_affine)
        nib.save(importance_nii, results_dir_maps + f'{measure}_{model_name}_importance_map.nii.gz')

#### Surface plots (Figure 4)

In [ ]:
measure='ECM_rank'
model='LDA'
# nii_file = results_dir_maps + measure + '_Logistic Regression_importance_map.nii.gz'  
nii_file = results_dir_maps + measure + '_LDA_importance_map.nii.gz'  

stat_img = image.load_img(nii_file)
data = stat_img.get_fdata()
min_val = np.nanmin(data)
max_val = np.nanmax(data)
print(min_val, max_val)

In [ ]:
min_val=-0.003
max_val=0.0025

fsaverage = datasets.fetch_surf_fsaverage("fsaverage")
texture_left = surface.vol_to_surf(stat_img, fsaverage.pial_left)
texture_right = surface.vol_to_surf(stat_img, fsaverage.pial_right)

colors = [
    (0.00, 0.20, 0.60),  
    (1.00, 1.00, 0.80), 
    (0.80, 0.00, 0.00),  
]

colormap = LinearSegmentedColormap.from_list(
    "colormap", colors
)

fig =plotting.plot_surf_stat_map(
    surf_mesh=fsaverage.infl_left,
    stat_map=texture_left,
    hemi='left',
    view='ventral',
    cmap=colormap,
    symmetric_cbar=False,
    colorbar=True,
    darkness=None,
    bg_on_data=True,
    vmax=max_val,
    vmin=min_val
)
plt.savefig(figures_dir+measure+'_'+model+'_ventral_left.png', dpi=300 )

fig =plotting.plot_surf_stat_map(
    surf_mesh=fsaverage.infl_left,
    stat_map=texture_left,
    hemi='left',
    cmap=colormap,
    symmetric_cbar=False,
    colorbar=True,
    darkness=None,
    bg_on_data=True,
    vmax=max_val,
    vmin=min_val

)
plt.savefig(figures_dir+measure+'_'+model+'_left.png', dpi=300 )

fig =plotting.plot_surf_stat_map(
    surf_mesh=fsaverage.infl_left,
    stat_map=texture_left,
    hemi='right',
    view='lateral',
    cmap=colormap,
    symmetric_cbar=False,
    colorbar=True,
    darkness=None,
    bg_on_data=True,
    vmax=max_val,
    vmin=min_val

)
plt.savefig(figures_dir+measure+'_'+model+'_left_lateral.png', dpi=300 )

fig =plotting.plot_surf_stat_map(
    surf_mesh=fsaverage.infl_right,
    stat_map=texture_right,
    hemi='right',
    view='ventral',
    cmap=colormap,
    symmetric_cbar=False,
    colorbar=True,
    darkness=None,
    bg_on_data=True,
    vmax=max_val,
    vmin=min_val

)
plt.savefig(figures_dir+measure+'_'+model+'_ventral_right.png', dpi=300 )

fig =plotting.plot_surf_stat_map(
    surf_mesh=fsaverage.infl_right,
    stat_map=texture_right,
    hemi='right',
    cmap=colormap,
    symmetric_cbar=False,
    colorbar=True,
    darkness=None,
    bg_on_data=True,
    vmax=max_val,
    vmin=min_val

)
plt.savefig(figures_dir+measure+'_'+model+'_right.png', dpi=300 )

fig =plotting.plot_surf_stat_map(
    surf_mesh=fsaverage.infl_right,
    stat_map=texture_right,
    hemi='left',
    view='lateral',
    cmap=colormap,
    symmetric_cbar=False,
    colorbar=True,
    darkness=None,
    bg_on_data=True,
    vmax=max_val,
    vmin=min_val
)
plt.savefig(figures_dir+measure+'_'+model+'_right_lateral.png', dpi=300 )